In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ford=pd.read_csv("ford.csv")
ford.info()

In [ ]:
fordData=pd.DataFrame(ford)
fordData.dropna(inplace=True)
fordData.describe(include='all')

In [ ]:
fuel_counts = fordData['fuelType'].value_counts()
fuel_counts

### fuelType sütununda yalnızca 'Petrol' ve 'Diesel' olmak üzere iki farklı türe indirdik. Bunun sebebi diğer türlerde çok az sayıda veri satırı olduğu için modelin doğruluk oranını düşürecekti.

In [ ]:
fordData.drop(fordData[fordData['fuelType'].isin(['Hybrid','Electric', 'Other'])].index, inplace=True)
fuel_counts = fordData['fuelType'].value_counts()
fuel_counts

### Veri setindeki 'fuelType' satırı string türünde veri içerdiği için bunları 0 ve 1 olmak üzere integer verilere dönüştürdük. Arkasında hiçbir matematik yok keyfi bir dönüştürme seçimi oldu.

Petrol -> 0
Diesel -> 1


In [ ]:
FuelTypeDict={'Diesel':1,'Petrol':0}
fordData['fuelType'] = fordData['fuelType'].map(FuelTypeDict)
fordData.head(10)

In [ ]:
TransType_counts = fordData['transmission'].value_counts()
TransType_counts

### Veri setindeki 'transmission' satırı string türünde veri içerdiği için bunları 0,1 ve 2 olmak üzere integer verilere dönüştürdük. Arkasında hiçbir matematik yok keyfi bir dönüştürme seçimi oldu.

Semi-Auto -> 0
Manual -> 1
Automatic -> 2

In [ ]:
TransmissionTypeDict={'Manual':1,'Automatic':2,'Semi-Auto':0}
fordData['transmission'] = fordData['transmission'].map(TransmissionTypeDict)
fordData.head(10)

In [ ]:
model_counts = fordData['model'].value_counts()

plt.bar(model_counts.index, model_counts.values)
plt.xticks(rotation=90)
plt.show()


 ##### Model türleri 4'e indirildi. Bunun sebebi ise diğer model türleri çok az sayıda veri satırı içeriyordu ve bu durum modelimizin doğruluk oranını düşürecekti. Kategori türlerinin ilk iki harfi ASCII kodlarına dönüştürüldü . Bunlar;
“Fiesta” için “FI” harfleri 70 ve 73 ASCII kodlarına dönüşecektir.

“Focus” için “FO” harfleri 70 ve 79 ASCII kodlarına dönüşecektir.

“Kuga” için “KU” harfleri 75 ve 85 ASCII kodlarına dönüşecektir.

“EcoSport” için “EC” harfleri 69 ve 67 ASCII kodlarına dönüşecektir.


In [ ]:
models_to_keep = [' Fiesta', ' Focus', ' Kuga', ' EcoSport']
fordData = fordData[fordData['model'].isin(models_to_keep)]
model_to_ascii={' Fiesta':7073,' Focus':7079,' Kuga':7585,' EcoSport':6967}
fordData['model'] = fordData['model'].map(model_to_ascii)
model_counts = fordData['model'].value_counts()
model_counts

### Yıl grafiğinde bazı yıllara ait çok az sayıda veri vardı bu modelin doğruluk oranını düşüreceği için o veri satırlarını veri setinin içinden atmamız lazım!

In [ ]:
year_counts = fordData['year'].value_counts()

plt.bar(year_counts.index, year_counts.values)
plt.xticks(rotation=90)
plt.show()

##### Bu şekilde verileri en çok dağılım gösterdiği yıllar arasında yer alan verilere indirgemiş olduk.

In [ ]:
fordData = fordData[(fordData['year'] >= 2013) & (fordData['year'] < 2020)]
year_counts = fordData['year'].value_counts()

plt.bar(year_counts.index, year_counts.values)
plt.xticks(rotation=90)
plt.show()

### Dataframe'in model tarafından işlenebilir hale getirdikten sonra, 'fordData_Refined.csv' isimli dosyayı oluşturup  içerisine yazdırdık.

In [ ]:
fordData.to_csv('fordData_Refined.csv',index=False)
fordData.describe(include='all')

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

In [40]:
X = fordData[['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']]
y = fordData['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Ortalama Mutlak Hata: {mae}')
print(f'Ortalama Kare Hata: {mse}')
print(f'R^2 Skoru: {r2}')

In [ ]:
mlp_model = MLPRegressor(hidden_layer_sizes=(80, 70), max_iter=300, random_state=42)
mlp_model.fit(X_train_scaled, y_train)
y_pred = mlp_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Ortalama Mutlak Hata: {mae}')
print(f'Ortalama Kare Hata: {mse}')
print(f'R^2 Skoru: {r2}')

In [43]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore

In [ ]:

model = Sequential()

model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

model.fit(X_train_scaled, y_train, epochs=100, batch_size=30, validation_split=0.2, verbose=1)

y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Ortalama Mutlak Hata: {mae}')
print(f'Ortalama Kare Hata: {mse}')
print(f'R^2 Skoru: {r2}')
